In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
from transformers import AutoModel
model = AutoModel.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
model.to(device)

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

2025-06-13 14:35:02.973816: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749825303.178682      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749825303.236661      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

Setup Data

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

dataset = load_dataset("parquet", data_files={
    "train": "/kaggle/input/disease/train-00000-of-00001-baac38b53532b0da.parquet",
    "test": "/kaggle/input/disease/test-00000-of-00001-1019821dbb200a34.parquet"
})

label_list = ["O", "B-Disease", "I-Disease"]
label_to_id = {label: i for i, label in enumerate(label_list)}

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=128
    )

    labels = []
    for i, label_seq in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label_seq[word_idx])  
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

Map:   0%|          | 0/15488 [00:00<?, ? examples/s]

Map:   0%|          | 0/5737 [00:00<?, ? examples/s]

In [4]:
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

from torch.utils.data import DataLoader

train_loader = DataLoader(tokenized_dataset["train"], batch_size=16, shuffle=True)
test_loader = DataLoader(tokenized_dataset["test"], batch_size=16)

In [ ]:
from transformers import AutoModel
import torch.nn as nn
from torchinfo import summary

class DiseaseNER(nn.Module):
    def __init__(self, num_labels):
        super(DiseaseNER, self).__init__()
        self.bert = AutoModel.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        sequence_output = outputs[0] 
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output) 
        return logits
model= DiseaseNER(3).to(device)

atch_size = 2
seq_len = 128
dummy_input_ids = torch.randint(0, model.bert.config.vocab_size, (16, seq_len)).to(device)
dummy_attention_mask = torch.ones((16, seq_len), dtype=torch.long).to(device)

summary(model=model,
        input_data=(dummy_input_ids, dummy_attention_mask),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])


Layer (type (var_name))                                                Input Shape          Output Shape         Param #              Trainable
DiseaseNER (DiseaseNER)                                                [16, 128]            [16, 128, 3]         --                   True
├─BertModel (bert)                                                     [16, 128]            [16, 768]            --                   True
│    └─BertEmbeddings (embeddings)                                     --                   [16, 128, 768]       --                   True
│    │    └─Embedding (word_embeddings)                                [16, 128]            [16, 128, 768]       22,268,928           True
│    │    └─Embedding (token_type_embeddings)                          [16, 128]            [16, 128, 768]       1,536                True
│    │    └─Embedding (position_embeddings)                            [1, 128]             [1, 128, 768]        393,216              True
│    │    └─LayerNorm 

In [ ]:
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
def train_step(model: nn.Module, dataloader: DataLoader,  loss_fn: nn.CrossEntropyLoss,optimizer: torch.optim.Optimizer, device: torch.device):
    model.train()
    total_loss = 0

    for batch in tqdm(dataloader, desc="Training", leave=True):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(batch['input_ids'], batch['attention_mask']) 
        loss = loss_fn(outputs.view(-1, outputs.shape[-1]), batch['labels'].view(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

def eval_step(model, dataloader, loss_fn, device):
    model.eval()
    total_loss = 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(batch['input_ids'], batch['attention_mask']) 
            loss = loss_fn(outputs.view(-1, outputs.shape[-1]), batch['labels'].view(-1))
            total_loss += loss.item()

            preds = torch.argmax(outputs, dim=-1).view(-1)
            labels = batch['labels'].view(-1)

            mask = labels != -100
            all_preds.extend(preds[mask].cpu().numpy())
            all_labels.extend(labels[mask].cpu().numpy())

    report = classification_report(all_labels, all_preds, output_dict=True, zero_division=0)
    return total_loss / len(dataloader), report

In [7]:
from torch import nn
import torch.optim as optim
from transformers import get_scheduler
def train_model(model, train_loader, test_loader, device, epochs=3, lr=2e-5):
    model.to(device)

    optimizer = optim.AdamW(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss(ignore_index=-100)

    scheduler = get_scheduler(
        name="linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=epochs * len(train_loader)
    )

    for epoch in tqdm(range(epochs)):
        print(f"Epoch {epoch+1}/{epochs}")

        train_loss = train_step(model, train_loader, loss_fn, optimizer, device)
        test_loss, test_report = eval_step(model, test_loader, loss_fn, device)

        scheduler.step()

        print(f"Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f}")
        print(f"F1-score: {test_report['macro avg']['f1-score']:.4f}")
        print(f"Recall: {test_report['macro avg']['recall']:.4f}")
        print(f"Precision: {test_report['macro avg']['precision']:.4f}")


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DiseaseNER(num_labels=3)

train_model(model, train_loader, test_loader, device, epochs=3)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1/3


Training:   0%|          | 0/968 [00:00<?, ?it/s]

Train Loss: 0.0707 | Test Loss: 0.0454
F1-score: 0.9005
Recall: 0.9294
Precision: 0.8746
Epoch 2/3


Training:   0%|          | 0/968 [00:00<?, ?it/s]

Train Loss: 0.0262 | Test Loss: 0.0407
F1-score: 0.9067
Recall: 0.9104
Precision: 0.9031
Epoch 3/3


Training:   0%|          | 0/968 [00:00<?, ?it/s]

Train Loss: 0.0146 | Test Loss: 0.0574
F1-score: 0.8972
Recall: 0.9288
Precision: 0.8705
